In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
# list of site names
sites_list = spacex_df['Launch Site'].unique().tolist()
sites_list

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [4]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': sites_list[0], 'value': sites_list[0]},
                                        {'label': sites_list[1], 'value': sites_list[1]},
                                        {'label': sites_list[2], 'value': sites_list[2]},
                                        {'label': sites_list[3], 'value': sites_list[3]},
                                    ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0',
                                          2500: '2500',
                                          5000: '5000',
                                          7500: '7500',
                                          10000: '10000'},
                                    value=[2000, 8000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        data_rate = spacex_df[['Launch Site', 'class']].groupby(['Launch Site']).mean().reset_index()
        fig = px.pie(data_rate, values='class', 
            names='Launch Site', 
            title='Total success launches by site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_rate = site_df['class'].value_counts(normalize=True).reset_index()
        fig = px.pie(site_rate, values='proportion', 
            names='class', 
            title='Success rate for site:' + entered_site)
        return fig
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, payload_mass):
    if entered_site == 'ALL':
        allsites_mass_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])]
        fig = px.scatter(allsites_mass_df, x='Payload Mass (kg)', y='class', color="Booster Version Category",
            title='Correlation between payload and success for all sites')
        return fig
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_mass_df = site_df[site_df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])]
        fig = px.scatter(site_mass_df, x='Payload Mass (kg)', y='class', color="Booster Version Category",
                    title='Correlation between payload and success for site ' + entered_site)
        return fig
        
# Run the app
if __name__ == '__main__':
    app.run()